In [1]:
from PPOAgent import PPOAgent
from SubEnvironment import SubEnvironment
from keras import backend as K
%load_ext autoreload
%autoreload 2

In [2]:
env = SubEnvironment(max_steps=512, period=128, lookback=64,
                     frequency=(1,10), freq_var=(0,1.),
                     amplitude=1., amp_var=(0., 0.5),
                     noise=0.1, phase=(0,1))

In [28]:
K.clear_session()
with PPOAgent(env, verbose=1) as agent:
    agent.run(10)

2019-01-07 20:41:55,513 Run Logs Created: ./runs/agent-26/
2019-01-07 20:41:55,514 Starting run: agent v26
2019-01-07 20:41:56,311 Episode 0 done! Reward: -17.212179577857274
2019-01-07 20:41:58,295 Episode 1 done! Reward: -45.55260055143596
2019-01-07 20:41:59,191 Episode 2 done! Reward: -20.97988037362107
2019-01-07 20:42:00,077 Episode 3 done! Reward: -10.97721403061995
2019-01-07 20:42:00,974 Episode 4 done! Reward: 2.232171726978299
2019-01-07 20:42:01,885 Episode 5 done! Reward: -17.648861611310302
2019-01-07 20:42:02,808 Episode 6 done! Reward: -19.552187963264075
2019-01-07 20:42:03,711 Episode 7 done! Reward: -20.434188867993466
2019-01-07 20:42:04,581 Episode 8 done! Reward: -18.100897642812072
2019-01-07 20:42:05,496 Episode 9 done! Reward: -40.1694568571358


In [20]:
import time

time.time()

1546818553.8979425

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gym
from copy import deepcopy

from keras.layers import Input, Dense, concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras import backend as K

from SubAgent import SubAgent
from SubEnvironment import SubEnvironment
from MetaAgent import MetaAgent
from MetaEnvironment import MetaEnvironment

K.clear_session()

%load_ext autoreload
%autoreload 2

import os
import threading
from IPython.display import display
import ipywidgets as widgets
import time
import multiprocessing
from sidecar import Sidecar
jobs = []

In [2]:
sub_envs = []
for i in range(3):
    sub_envs.append(SubEnvironment(max_steps=512, period=128, lookback=64,
                                   frequency=(1,10), freq_var=(0,1.),
                                   amplitude=1., amp_var=(0., 0.5),
                                   noise=0.1, phase=(0,1)))
    
sub_agents = []
for env in sub_envs:
    sub_agents.append(SubAgent(env))
    
meta_env = MetaEnvironment(sub_agents)

Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf.print executes in graph mode:
```python
    sess = tf.Session()
    with sess.as_default():
        tensor = tf.range(10)
        print_op = tf.print(tensor)
        with tf.control_dependencies([print_op]):
          out = tf.add(tensor, tensor)
        sess.run(out)
    ```
Additionally, to use tf.print in python 2.7, users must make sure to import
the following:

  `from __future__ import print_function`



In [ ]:
meta_agent = MetaAgent(meta_env, live_plot=False)
def run_agent():
    return meta_agent.run(episodes=128)

p = multiprocessing.Process(target=run_agent)
jobs.append(p)
p.start()

In [16]:
import os

In [ ]:
os.makedirs('.', exist_ok=True)

In [ ]:
run_dir = "./runs/run-25/"
frame_dir = run_dir + "animations/meta_plot/"
frame_count = len(os.listdir(frame_dir)) - 1
line_limit = 10
log_path = run_dir + 'log.txt'
log_val = []
kill_thread = False
last_frame_count = 0
empty_frame_count = 0

def update_log():
    log_val = ['<li><code style="font-size: 10px;">{}</code></li>'.format(line) for 
               line in list(open(log_path))[-line_limit:]]
    log_widget.value = '<ul style="list-style-type: none; padding: 0;">' + ''.join(log_val) + '</ul>'
    
def update_frames():
    frame_count = len(os.listdir(frame_dir)) - 1
    play_widget.max = frame_count
    slider_widget.max = frame_count
    if play_widget.value == play_widget.max - 1:
        play_widget.value = play_widget.max
    
def update_image(i=0):
    file = open(frame_dir + "{0:03d}.png".format(i), "rb")
    image = file.read()
    image_widget.value = image

def on_change(x):
    if (x['name'] == 'value'):
        update_image(x['new'])
        
def kill_thread_click(x):
    global kill_thread
    kill_thread = True
        
def live_update():
    global last_frame_count
    global empty_frame_count
    while True:
        update_count_widget.value = "Last Update: " + time.strftime('%c')
        update_log()
        update_frames()
        if last_frame_count == frame_count:
            empty_frame_count += 1
        else:
            last_frame_count = frame_count
            empty_frame_count = 0
        if empty_frame_count > 10 or kill_thread:
            update_count_widget.value = 'Halted: ' + time.strftime('%c')
            break
        time.sleep(1)

play_widget = widgets.Play(
    interval=250,
    value=frame_count,
    min=0,
    max=frame_count,
    step=1,
    description="Press play",
    disabled=False
)
slider_widget = widgets.IntSlider(max=frame_count)
image_widget = widgets.Image(format='png', height=100)
 
slider_widget.observe(on_change)
widgets.jslink((play_widget, 'value'), (slider_widget, 'value'))

log_widget = widgets.HTML()

update_count_widget = widgets.Label()

kill_button_widget = widgets.Button(
    description='Kill Thread'
)
kill_button_widget.on_click(kill_thread_click)

thread = threading.Thread(target=live_update)
thread.start()

training_info = widgets.VBox([image_widget,
                      widgets.HBox([kill_button_widget, play_widget, slider_widget, update_count_widget]),
                      log_widget])

sc = Sidecar(title='Training Info')
with sc:
    display(training_info)

In [114]:
loggers = {
    'one': 1,
    'two': 2,
    'three': 3
}

In [116]:
'one' in loggers

True